# Get Randomized Audience Cohorts From Inference

In [1]:
%load_ext autoreload
%autoreload 2

::: {.content-hidden}
Import necessary Python modules
:::

In [2]:
import os
import sys
from calendar import month_name
from datetime import datetime

import mlflow.sklearn
import pandas as pd

::: {.content-hidden}
Get relative path to project root directory
:::

In [3]:
PROJ_ROOT_DIR = os.path.join(os.pardir)
src_dir = os.path.join(PROJ_ROOT_DIR, "src")

sys.path.append(src_dir)

::: {.content-hidden}
Import custom Python modules
:::

In [4]:
#| output: false
%aimport audience_size_helpers
import audience_size_helpers as ash

%aimport bigquery_auth_helpers
from bigquery_auth_helpers import auth_to_bigquery

%aimport cohorts
import cohorts as ch

%aimport data_checks_helpers
import data_checks_helpers as dch

%aimport model_helpers
import model_helpers as modh

%aimport profile_audience
import profile_audience as pa

%aimport sql_helpers
import sql_helpers as sqlh

%aimport transform_helpers
import transform_helpers as th

%aimport utils
import utils as ut

/opt/conda/envs/train/lib/python3.11/site-packages/umap/distances.py:1063: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/opt/conda/envs/train/lib/python3.11/site-packages/umap/distances.py:1071: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/opt/conda/envs/train/lib/python3.11/site-packages/umap/distances.py:1086: NumbaDeprecationWarni

## About

### Overview

This step generates the randomized test and control cohorts needed to run a marketing campaign. The impact of the campaign on conversions (KPI) will be assessed using an A/B test at the end of the campaign. The randomized cohorts are needed in order to conduct this test. This is step 4. from a [typical A/B Testing workflow](https://www.datacamp.com/blog/data-demystified-what-is-a-b-testing).

### Order of Operations
This step can be run prospectively at the end of the inference period, just before the start of the campaign, when all the inference data (first-time visitors to the store) becomes available.

For the current use-case, the required sizes of one or more audience groups have been determined in the previous step. Recall that an audience strategy determines if one of more groups are used. For a strategy consisting of a single audience group, only the visitors predicted to have a high propensity to make a purchase on a return visit are selected to participate in the campaign and so randomized cohorts are to be drawn from this single group. For multiple audience groups, namely visitors predicted to have a low, medium or high propensity, randomized cohorts are to be drawn from each such group.

With this in mind, this step first assigns all first-time visitors to the store during the inference period to an audience group based on the requied audience strategy (single or multiple audience groups).

Next, before generating audience cohorts, features in the unseen (inference) data are checked for data drift relative to the test data used during ML model development.

1. Use statistical tests, that are called as part of EvidentlyAI's data drift monitor ([1](https://www.evidentlyai.com/blog/tutorial-3-historical-data-drift), [2](https://github.com/evidentlyai/evidently/blob/main/examples/integrations/mlflow_logging/historical_drift_visualization.ipynb)) [for tabular data](https://docs.evidentlyai.com/user-guide/customization/options-for-statistical-tests#tabular-drift-detection). The following [statistical tests and logic are used by EvidentlyAI](https://docs.evidentlyai.com/reference/data-drift-algorithm#tabular-data) and are also implemented here
   - numerical features with less than or equal to 1,000 observations (this is not used here since the inference data consists of more than 1,000 observations during the inference data period)
     - [Kolmogorov-Smirnov goodness-of-fit Test (two-sample version)](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.kstest.html#scipy.stats.kstest)
   - numerical features with more than 1,000 observations (this is the case here since the inference data has more than 1,000 first-time visitors during the inference data period)
     - [Wasserstein distance](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.wasserstein_distance.html#scipy.stats.wasserstein_distance)
   - categorical features with less than or equal to 1,000 observations (not used here)
     - [chi-squared goodness-of-fit test](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.chi2.html#scipy.stats.chi2)
   - categorical features with more than 1,000 observations (used here)
     - [Jensen-Shannon distance](https://docs.scipy.org/doc/scipy/reference/generated/scipy.spatial.distance.jensenshannon.html#scipy.spatial.distance.jensenshannon)

   Here, a custom function is defined to manually implement these tests on the unseen (inference) data.
3. Perform sanity checks on the data, by directly using EvidentlyAI's data test suite ([1](https://docs.evidentlyai.com/get-started/tutorial#7.-run-data-stability-tests), [2](https://docs.evidentlyai.com/user-guide/tests-and-reports/run-tests#using-test-presets)). A suite consisting of the following tests is applied to the unseen (inference) data in order to test that the number of
   - columns with missing values (columns with at least one missing value)
   - rows with missing values 
   - constant (single-valued) columns
   - duplicated rows (rows that are duplicates)
   - duplicated columns (columns that are duplicates)
   - column datatypes
   - empty columns (all values in a column are missing)
   - empty rows (all values in a row are missing)
   - rows with missing values (rows with at least one missing value)
   - number of columns

   in the unseen (inference) data equals the number in the reference data (test split from the data used during ML model development).

Finally, within each audience group, visitors are randomly assigned to test or control cohorts. A brief profile of each audience group is then performed using attributes of the visitors' first visit. The profile and the inference data with the audience group and cohort for each first-time visitor during the inference period can be used by the marketing team to design and implement the marketing campaign aimed at growing the customer base.

### Implementation

A custom python module in `src/cohorts.py` has been developed to create the test and control audience cohorts based on required sample sizes that were estimated in the previous step and logged as a MLFlow artifact (file) for the best MLFlow deployment candidate model.

The procedure followed in this step is briefle outlined below

1. the MLFlow artifact (file) is loaded in order to access the required sample sizes for each
   - audience strategy
   - desired combination of effect sizes (power, confidence level and uplift)
2. the file is filtered based on the
   - audience strategy
   - desired combination of effect sizes (see `wanted_effect_sizes` from **User Inputs**)
3. inference data (first-time visitors) is retrieved and the best ML model is used to make inference predictions of the propensity of these visitors to make a purchase on a return (future) visit to the store.
4. propensities are used to assign first-time visitors to audience groups (or bins)
   - if the desired audience strategy is for a single audience group, then visitors are placed into a single bin
   - if the desired audience strategy is for multiple audience groups, then multiple bins are creaed
5. for each bin (audience group), visitors are randomly placed into test and control cohorts
6. the inference data with
   - predicted propensity (probability)
   - audience group
   - cohort (test or control)

   is then logged as a MLFlow artifact for use by the marketing team to build and design the campaign.

## User Inputs

Define the following

1. start and end dates for inference data
2. name of column containing label (outcome)
3. list of categorical features
4. list of numerical features, including categorical features present in the raw data as integers
5. `audience_groups`
   - desired audience groups into which the first-time visitors propensities will be placed
     - `num_propens_groups` specifies the number of desired audience propensity groups (low, medium and high)
     - `propens_group_labels` specifies names of the desired audience propensity groups
6. inputs (uplift, powerm confidence level) for which random cohort sizes are to be created
7. type of audience strategy (single- or multi- group) from which to create cohorts

In [5]:
#| echo: true
# 1. start and end dates
infer_start_date = "20170301"
infer_end_date = "20170331"

# 2. label column
label = "made_purchase_on_future_visit"

# 3. categorical column
categoricals=[
    'bounces',
    'source',
    'medium',
    'channelGrouping',
    'last_action',
    'browser',
    'os',
    'deviceCategory',
]

# 4. numerical columns
numericals=[
    'hits',
    'pageviews',
    'promos_displayed',
    'product_views',
    'product_clicks',
    'time_on_site',
]

# 5. mapping dictionaries
audience_groups_strategy_1 = {
    "num_propens_groups": 3,
    "propens_group_labels": ["High", "Medium", "Low"],
}
audience_groups_strategy_2 = {
    "num_propens_groups": 3,
    "propens_group_labels": ["High", "High-Medium", "High-Medium-Low"],
}

# 6. wanted inputs for estimating sample sizes
wanted_inputs = {
    "uplift_percentage": 10,
    "power_percentage": 55,
    "confidence_level_percentage": 55,
}

# 7. type of audience strategy to use when creating groups
audience_strategy = 1

::: {.content-hidden}
Get path to data sub-folders
:::

In [6]:
data_dir = os.path.join(PROJ_ROOT_DIR, "data")
raw_data_dir = os.path.join(data_dir, "raw")
processed_data_dir = os.path.join(data_dir, "processed")

Create a mapping between audience group number (0, 1, 2) and group name

In [7]:
#| echo: true
mapper_dict_audience_strategy_1 = dict(
    zip(
        range(audience_groups_strategy_1["num_propens_groups"]),
        audience_groups_strategy_1["propens_group_labels"],
    )
)
mapper_dict_audience_strategy_2 = dict(
    zip(
        range(audience_groups_strategy_2["num_propens_groups"]),
        audience_groups_strategy_2["propens_group_labels"],
    )
)
print(mapper_dict_audience_strategy_1)
print(mapper_dict_audience_strategy_2)

{0: 'High', 1: 'Medium', 2: 'Low'}
{0: 'High', 1: 'High-Medium', 2: 'High-Medium-Low'}


Get desired effect size queries and number of audience groups

In [8]:
#| echo: true
if audience_strategy == 1:
    query_inputs = (
        f"(uplift == {wanted_inputs['uplift_percentage']}) & "
        f"(power == {wanted_inputs['power_percentage']}) & "
        f"(ci_level == {wanted_inputs['confidence_level_percentage']})"
    )
else:
    query_inputs = (
        f"(group_size_proportion < 34) & "
        f"(uplift == {wanted_inputs['uplift_percentage']}) & "
        f"(power == {wanted_inputs['power_percentage']}) & "
        f"(ci_level == {wanted_inputs['confidence_level_percentage']})"
    )

num_bins = (
    audience_groups_strategy_1["num_propens_groups"]
    if audience_strategy == 1
    else audience_groups_strategy_2["num_propens_groups"]
)

::: {.content-hidden}
Define MLFlow storage paths
:::

In [9]:
mlruns_db_fpath = f"{raw_data_dir}/mlruns.db"
mlflow.set_tracking_uri(f"sqlite:///{mlruns_db_fpath}")

::: {.content-hidden}
Set environment variable to silence MLFlow `git` warning messsage
:::

In [10]:
os.environ["GIT_PYTHON_REFRESH"] = "quiet"

::: {.content-hidden}
Define a dictionary to specify datatypes of the transformed *inference* data
:::

In [11]:
dtypes_dict = {
    "fullvisitorid": pd.StringDtype(),
    "visitId": pd.StringDtype(),
    "visitNumber": pd.Int8Dtype(),
    "quarter": pd.Int8Dtype(),
    "month": pd.Int8Dtype(),
    "day_of_month": pd.Int8Dtype(),
    "day_of_week": pd.Int8Dtype(),
    "hour": pd.Int8Dtype(),
    "minute": pd.Int8Dtype(),
    "second": pd.Int8Dtype(),
    "source": pd.CategoricalDtype(),
    "medium": pd.CategoricalDtype(),
    "channelGrouping": pd.CategoricalDtype(),
    "hits": pd.Int16Dtype(),
    "bounces": pd.CategoricalDtype(),
    "last_action": pd.CategoricalDtype(),
    "promos_displayed": pd.Int16Dtype(),
    "promos_clicked": pd.Int16Dtype(),
    "product_views": pd.Int16Dtype(),
    "product_clicks": pd.Int16Dtype(),
    "pageviews": pd.Int16Dtype(),
    "time_on_site": pd.Int16Dtype(),
    "browser": pd.CategoricalDtype(),
    "os": pd.CategoricalDtype(),
    "added_to_cart": pd.Int16Dtype(),
    "deviceCategory": pd.CategoricalDtype(),
    "made_purchase_on_future_visit": pd.Int8Dtype(),
}

::: {.content-hidden}
Create a mapping between action type integer and label, in order to get meaningful names from the `action_type` column
:::

In [12]:
action_mapper = {
    1: "Click through of product lists",
    2: "Product detail views",
    3: "Add product(s) to cart",
    4: "Remove product(s) from cart",
    5: "Check out",
    6: "Completed purchase",
    7: "Refund of purchase",
    8: "Checkout options",
    0: "Unknown",
}

::: {.content-hidden}
## Authenticate to `BigQuery`
:::

In [13]:
gcp_auth_dict = auth_to_bigquery(raw_data_dir)

## Get Inference Data

In [14]:
#| echo: true
query_infer = sqlh.get_sql_query_infer(infer_start_date, infer_end_date)
X_infer, _ = th.extract_data(query_infer, gcp_auth_dict).pipe(
    th.transform_data,
    datatypes_dict={k:v for k,v in dtypes_dict.items() if k != label},
    duplicate_cols=["fullvisitorid"],
    column_mapper_dict={'last_action': action_mapper},
)
X_infer = X_infer.pipe(th.shuffle_data)

Query execution start time = 2023-06-05 19:14:19.839...done at 2023-06-05 19:14:25.363 (5.524 seconds).
Query returned 21,768 rows
Got 21,752 rows and 27 columns after dropping duplicates
Transformed data has 21,752 rows & 27 columns


Get the size of each audience cohort in the inference data

In [15]:
#| echo: true
bin_size_infer = len(X_infer) / num_bins
bin_size_infer_control = int(bin_size_infer / 2)

## Get Model

### Fetch Latest Version of Best Deployment Candidate Model

Get name of best deployment candidate model from MLFlow model registry

In [16]:
#| echo: true
df_deployment_candidate_mlflow_models = modh.get_all_deployment_candidate_models()
best_run_model_name = modh.get_best_deployment_candidate_model(
    df_deployment_candidate_mlflow_models
)
with pd.option_context("display.max_colwidth", None):
    display(df_deployment_candidate_mlflow_models)

,name,description,run_id,tags,version,score
0,BetaDistClassifier_20160901_20170228_133892_feats__20230605_225228,Best Model based on fbeta2 score of 0.4985941947,742cf152d5094be0b686edf910c7d398,{'deployment-candidate': 'yes'},2,0.498594


::: {.content-hidden}
Load best deployment candidate model object
:::

In [17]:
model = mlflow.sklearn.load_model(model_uri=f"models:/{best_run_model_name}/latest")
model

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('aboveavg',
                                                                   AboveAveragePagePromoEngager(cols=['hits',
                                                                                                      'promos_displayed',
                                                                                                      'promos_clicked',
                                                                                                      'product_views',
                                                                                                      'product_clicks',
                                                                                                      'pageviews',
                                                                                                      'time_on_site'])),
                                                                  ('scaler',
                                                                   MinMaxScaler())]),
                                                  ['hits', 'promos_displayed',
                                                   'promos_clicked',
                                                   'product_views',
                                                   'product_clicks',
                                                   'page...
                                                                  ('dummy',
                                                                   OneHotEncoder(drop='first',
                                                                                 dtype=<class 'int'>,
                                                                                 handle_unknown='ignore'))]),
                                                  ['bounces', 'last_action',
                                                   'source', 'medium',
                                                   'channelGrouping', 'browser',
                                                   'os', 'deviceCategory'])])),
                ('select', DropCorrelatedFeatures(threshold=0.7)),
                ('resampler',
                 RandomOverSampler(random_state=88, sampling_strategy=0.1)),
                ['clf', BetaDistClassifier(a=0.3, b=2.5, random_state=88)]])

### Get Data Used to Develop Best Deployment Candidate Model

Get all available data used during model development of the best deployment candidate model

In [18]:
#| echo: true
df_all = modh.get_data_for_run_id(df_deployment_candidate_mlflow_models, 'processed_data')

::: {.content-hidden}
Perform sanity checks on features in inference data
:::

In [19]:
modh.check_data_per_best_expt_run(
    df_deployment_candidate_mlflow_models,
    df_all.drop(columns=["score", "predicted_label", "predicted_score_label", "split_type"]),
    X_infer,
    label,
)

### Check Data Drift and Stability

Get test split from the data used during development of the best deployment candidate model. This was the last month of data used during ML model development and it will be considered as the reference data.

In [20]:
#| echo: true
X_test_best_run = (
    df_all
    .query("split_type == 'test'")
    .drop(columns=['split_type', label])
)

In [21]:
#| output: false
with pd.option_context("display.max_columns", None):
    display(X_test_best_run.head(2))
    display(X_test_best_run.tail(2))

,fullvisitorid,visitId,visitNumber,visitStartTime,quarter,month,day_of_month,day_of_week,hour,minute,second,source,medium,channelGrouping,hits,bounces,last_action,promos_displayed,promos_clicked,product_views,product_clicks,pageviews,time_on_site,browser,os,deviceCategory,added_to_cart,score,predicted_score_label,predicted_label
113728,0092786413416632932,1487747471,1,2017-02-21 23:11:11,1,2,21,3,23,11,11,google,organic,Organic Search,4,0,Unknown,9,0,2,0,4,29,Chrome,Macintosh,desktop,0,0.000119,True,0
113729,9246024261954442050,1487316329,1,2017-02-16 23:25:29,1,2,16,5,23,25,29,siliconvalley.about.com,referral,Referral,4,0,Unknown,9,0,3,0,4,29,Chrome,Android,mobile,0,0.012315,True,0


,fullvisitorid,visitId,visitNumber,visitStartTime,quarter,month,day_of_month,day_of_week,hour,minute,second,source,medium,channelGrouping,hits,bounces,last_action,promos_displayed,promos_clicked,product_views,product_clicks,pageviews,time_on_site,browser,os,deviceCategory,added_to_cart,score,predicted_score_label,predicted_label
133890,91736817749198966,1488142038,1,2017-02-26 12:47:18,1,2,26,1,12,47,18,google,organic,Organic Search,3,0,Unknown,0,0,27,0,3,76,Chrome,Chrome OS,desktop,0,0.000048,True,0
133891,4990910589525036851,1486257216,1,2017-02-04 17:13:36,1,2,4,7,17,13,36,youtube.com,referral,Social,3,0,Unknown,0,0,30,0,3,91,Edge,Windows,desktop,0,0.000021,True,0


Check numerical features for drift between

1. ML development data (test split)
2. unseen data (inference)

using the Wasserstein test

In [22]:
#| echo: true
df_num_drift, _ = dch.detect_features_drift(
    X_infer,
    X_test_best_run,
    numericals=numericals,
    categoricals=[],
    kst_threshold=0.05,
    kst_params=dict(
        method='exact',
        N=max(len(X_test_best_run), len(X_infer)),
        alternative='two-sided',
    ),
    wsd_threshold=1,
)
display(df_num_drift)

,feature_type,feature,nunique_ref,nunique_curr,pct_diff_mean,pct_diff_std,abs_diff_mean,abs_diff_std,metric_value,metric_threshold,drift_detected,test_type
0,numerical,hits,104,112,-2.788887,-16.201796,-0.172507,-1.568929,0.222779,1,False,Wasserstein
1,numerical,pageviews,78,94,-2.221251,-19.111907,-0.117295,-1.415531,0.162935,1,False,Wasserstein
2,numerical,promos_displayed,21,18,-9.664244,2.995538,-0.754956,0.316693,0.786333,1,False,Wasserstein
3,numerical,product_views,273,286,1.210796,-5.643018,0.284302,-2.017041,0.721632,1,False,Wasserstein
4,numerical,product_clicks,30,35,-6.762275,-13.531232,-0.042843,-0.246337,0.042843,1,False,Wasserstein
5,numerical,time_on_site,1583,1702,-7.346253,-6.512765,-13.136182,-25.665576,13.500904,1,True,Wasserstein


::: {.callout-note title="Notes"}

1. For descriptive statistics, consider drift being present if both of the following conditions are met

   - `pct_diff_mean` > 5
     - percent difference between mean value in reference (test data) and current (inference) data is larger than 5%
   - `pct_diff_std` > 15
     - percent difference between standard deviation (which provides an indication of variability) in reference (test data) and current (inference) data is larger than 15%
2. Both percent differnces are calculated relative to the reference data.
:::

::: {.callout-tip title="Observations"}

1. The percent differences in the descriptive statistics (mean and standard deviation) suggest that drift is minimal for the following numerical features

   - `promos_displayed`
   - `product_views`
   - `time_on_site`
   - `product_clicks`
2. The `pageviews` and `hits` features show a drift from their higher variability, as suggested by their higher standard deviation, which in-turn indicates the presence of outliers. Both features do not show any drift in the mean percent difference.
3. The last four columns are related to the statistical test comparing the same feature in two datasets. A threshold of 1 was arbitrarily chosen. The limitation of this test is due to the difficulty in setting a reasonable threshold against which the metric can be compared. This test of drift could be useful if multiple inference periods were present during the production period. In such a scenario, the percent change in the metric could be tracked from one period to the next in order to determine if the feature is showing the presence of drift. These four columns are of less use in drift monitoring for a single infernce period.
:::

Perform sanity checks on all features between

1. ML development data (test split)
2. unseen data (inference)

In [23]:
#| echo: true
dtypes_eai = dict(zip(categoricals, ['str']*len(categoricals)))
df_stability = dch.check_data_stability(
    X_infer.astype(dtypes_eai),
    X_test_best_run.astype(dtypes_eai),
    numericals=numericals,
    categoricals=categoricals,
)
df_stability

Test suite start time = 2023-06-05 19:14:26.354...done at 2023-06-05 19:14:39.566 (13.212 seconds).


,all_passed,total_tests,success_tests,failed_tests,success,len_curr_data,len_ref_data,features_checked,num_features_checked
0,True,10,10,0,10,21752,20164,"[""bounces"", ""source"", ""medium"", ""channelGroupi...",14


::: {.callout-note title="Notes"}

1. The EvidentlyAI `TestSuite` module requires string features to have the `object` datatype. So, a datatype mapping dictionary was defined (`dtypes_eai`) to change the datatypes for these features.
:::

::: {.callout-tip title="Observations"}

1. It is reassuring that all data stability tests have passed. This indicates that features in the unseen (inference) data are stable in the 10 attributes listed in the **About** section, relative to those in the data used during ML development.
:::

## Make Inference Predictions

Make inference predictions with best deployment candidate model, using features extracted from the inference data

In [24]:
#| echo: true
#| output: false
y_infer_pred, y_infer_pred_proba = modh.make_inference(model, X_infer, label)
display(
    y_infer_pred.value_counts(normalize=True).rename('proportion').to_frame().merge(
        y_infer_pred.value_counts(normalize=False).rename('number').to_frame(),
        left_index=True,
        right_index=True,
        how='left',
    ).reset_index()
)

/opt/conda/envs/train/lib/python3.11/site-packages/sklearn/preprocessing/_encoders.py:202: UserWarning: Found unknown categories in columns [6] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
/opt/conda/envs/train/lib/python3.11/site-packages/sklearn/preprocessing/_encoders.py:202: UserWarning: Found unknown categories in columns [6] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


,made_purchase_on_future_visit,proportion,number
0,0,0.958762,20855
1,1,0.041238,897


::: {.callout-note title="Notes"}

1. Per the business use-case, these are predictions of whether a visitor will make a purchase during a later (return) visit to the merchandise store. Such predictions are made using attributes (features) of the first visit by visitors to the store and they can only be evaluated at a later time (after the outcome of the same visitor's later visit is known).
:::

## Create Cohorts

Perform the following to prepare the inference observations for extracting cohorts from the audience group(s)

1. combine inference features and predicted hard and soft labels into single `DataFrame`
2. sort predictions in ascending order of the predicted probability (propensity) (score)
3. separate the predictions into bins, using the predicted probability, based on the required number of bins
4. rename `bin_number` column (for use in downstream step)
5. set datatypes for columns

In [25]:
#| echo: true
df_infer_pred = (
    ch.combine_infer_data(X_infer, y_infer_pred, y_infer_pred_proba)
    .pipe(ash.sort_scores, False)
    .pipe(ash.get_audience_groups_by_propensity, num_bins)
    .pipe(ch.rename_columns, {"group_number": "maudience"})
    .pipe(
        ash.set_datatypes,
        {
            "row_number": pd.Int16Dtype(),
            "fullvisitorid": pd.StringDtype(),
            "score": pd.Float32Dtype(),
            "predicted_score_label": pd.BooleanDtype(),
            "maudience": pd.Int8Dtype(),
        },
    )
)
with pd.option_context("display.max_columns", 1000):
    display(df_infer_pred.dtypes.rename("dtype").to_frame().T)
    display(df_infer_pred)

Set all specified datatypes.


,fullvisitorid,visitId,visitNumber,visitStartTime,quarter,month,day_of_month,day_of_week,hour,minute,second,source,medium,channelGrouping,hits,bounces,last_action,promos_displayed,promos_clicked,product_views,product_clicks,pageviews,time_on_site,browser,os,deviceCategory,added_to_cart,score,predicted_score_label,row_number,maudience
dtype,string[python],string[python],Int8,datetime64[ns],Int8,Int8,Int8,Int8,Int8,Int8,Int8,category,category,category,Int16,category,category,Int16,Int16,Int16,Int16,Int16,Int16,category,category,category,Int16,Float32,boolean,Int16,Int8


,fullvisitorid,visitId,visitNumber,visitStartTime,quarter,month,day_of_month,day_of_week,hour,minute,second,source,medium,channelGrouping,hits,bounces,last_action,promos_displayed,promos_clicked,product_views,product_clicks,pageviews,time_on_site,browser,os,deviceCategory,added_to_cart,score,predicted_score_label,row_number,maudience
5667,0680963722765513682,1490246407,1,2017-03-22 22:20:07,1,3,22,4,22,20,7,google,organic,Organic Search,10,0,Product detail views,9,0,12,2,8,127,Chrome,Macintosh,desktop,0,0.955219,False,0,0
966,5015398141797883848,1488954219,1,2017-03-07 22:23:39,1,3,7,3,22,23,39,google,organic,Organic Search,20,0,Product detail views,9,0,163,1,19,663,Chrome,Windows,desktop,0,0.939487,False,1,0
2368,4256619380024888759,1490567835,1,2017-03-26 15:37:15,1,3,26,1,15,37,15,google,organic,Organic Search,1,1,Unknown,0,0,12,0,1,0,Chrome,Windows,desktop,0,0.932821,False,2,0
9009,3214011699506166608,1489445124,1,2017-03-13 15:45:24,1,3,13,2,15,45,24,(direct),(none),Direct,15,0,Completed purchase,9,0,16,1,13,414,Chrome,Chrome OS,desktop,1,0.932164,False,3,0
10507,0716469409246179583,1489155359,1,2017-03-10 06:15:59,1,3,10,6,6,15,59,google,organic,Organic Search,1,1,Unknown,0,0,12,0,1,0,Chrome,Windows,desktop,0,0.931829,False,4,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1271,2297134143716418806,1489463792,1,2017-03-13 20:56:32,1,3,13,2,20,56,32,google,organic,Organic Search,1,1,Unknown,0,0,0,0,1,0,Chrome,Android,mobile,0,0.0,False,21747,2
16990,4875088489598899615,1489358551,1,2017-03-12 15:42:31,1,3,12,1,15,42,31,google,cpc,Paid Search,8,0,Unknown,18,0,72,0,8,677,Chrome,Android,mobile,0,0.0,False,21748,2
832,504996867976621481,1488752321,1,2017-03-05 14:18:41,1,3,5,1,14,18,41,(direct),(none),Direct,12,0,Product detail views,18,0,37,1,11,898,Chrome,Chrome OS,desktop,0,0.0,False,21749,2
4867,3772471281235004535,1488495634,1,2017-03-02 15:00:34,1,3,2,5,15,0,34,youtube.com,referral,Social,3,0,Unknown,9,0,2,0,3,101,Internet Explorer,Windows,desktop,0,0.0,False,21750,2


::: {.callout-note title="Notes"}

1. This is the same data preparation that was used during the (previous) sample size estimation step step in order to prepare the test data split for estimating the required sample size.
:::

Perform the following to get the test and control cohorts from the inference data

1. load estimates for required sample sizes associated with best deployment candidate model
2. get required sample sizes for the chosen audience strategy
3. get required sample sizes that can be supported by size of inference data
4. get required sample sizes that support all required audience groups (bins)
5. get required sample sizes that capture required effect sizes
6. get random test and control cohorts

In [26]:
#| echo: true
df_infer_audience_groups = (
    ch.load_file_from_mlflow_artifact(
        df_deployment_candidate_mlflow_models, "audience_sample_sizes"
    )
    .pipe(ch.get_sample_sizes_by_strategy, audience_strategy)
    .pipe(ch.get_suitable_sample_sizes, bin_size_infer_control)
    .pipe(ch.get_sample_sizes_with_all_audience_groups, num_bins)
    .pipe(ch.get_required_inputs, query_inputs)
    .pipe(
        ch.create_cohorts,
        df_infer_pred,
        mapper_dict_audience_strategy_1
        if audience_strategy == 1
        else mapper_dict_audience_strategy_2,
        audience_strategy,
    )
    .pipe(
        ash.set_datatypes,
        {
            "maudience": pd.StringDtype(),
            "cohort": pd.StringDtype(),
            "audience_strategy": pd.Int8Dtype(),
        },
    )
)
with pd.option_context("display.max_columns", 1000):
    display(df_infer_audience_groups)

audience=0: High, size=7,251,  excluded=2,839, wanted=2,206, control=2,206, test=2,206
audience=1: Medium, size=7,250,  excluded=3,456, wanted=1,897, control=1,897, test=1,897
audience=2: Low, size=7,251,  excluded=3,045, wanted=2,103, control=2,103, test=2,103
Found suitable sample sizes and generated cohorts.
Set all specified datatypes.


,fullvisitorid,visitId,visitNumber,visitStartTime,quarter,month,day_of_month,day_of_week,hour,minute,second,source,medium,channelGrouping,hits,bounces,last_action,promos_displayed,promos_clicked,product_views,product_clicks,pageviews,time_on_site,browser,os,deviceCategory,added_to_cart,score,predicted_score_label,maudience,cohort,audience_strategy
0,0680963722765513682,1490246407,1,2017-03-22 22:20:07,1,3,22,4,22,20,7,google,organic,Organic Search,10,0,Product detail views,9,0,12,2,8,127,Chrome,Macintosh,desktop,0,0.955219,False,High,Control,1
1,3339620488023808675,1489208703,1,2017-03-10 21:05:03,1,3,10,6,21,5,3,(direct),(none),Direct,6,0,Product detail views,9,0,12,2,4,34,Safari,iOS,tablet,0,0.873791,False,High,Control,1
2,4352137968854306424,1489597665,1,2017-03-15 10:07:45,1,3,15,4,10,7,45,google,organic,Organic Search,2,0,Unknown,9,0,0,0,2,7,Safari,Macintosh,desktop,0,0.865694,False,High,Control,1
3,415961985355681274,1489621178,1,2017-03-15 16:39:38,1,3,15,4,16,39,38,(direct),(none),Direct,4,0,Unknown,9,0,39,0,4,90,Chrome,Windows,desktop,0,0.862547,False,High,Control,1
4,8333492141906527532,1490084582,1,2017-03-21 01:23:02,1,3,21,3,1,23,2,(direct),(none),Direct,1,1,Unknown,9,0,0,0,1,0,Safari,iOS,tablet,0,0.849589,False,High,Control,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21747,6565680845296579697,1490824341,1,2017-03-29 14:52:21,1,3,29,4,14,52,21,google,organic,Organic Search,1,1,Unknown,0,0,12,0,1,0,Chrome,Chrome OS,desktop,0,0.0,False,Low,<NA>,1
21748,536791133490221570,1490224587,1,2017-03-22 16:16:27,1,3,22,4,16,16,27,(direct),(none),Direct,4,0,Unknown,9,0,24,0,4,108,Chrome,Macintosh,desktop,0,0.0,False,Low,<NA>,1
21749,2449684517758558577,1489676669,1,2017-03-16 08:04:29,1,3,16,5,8,4,29,google,organic,Organic Search,2,0,Unknown,0,0,12,0,2,7,Safari,iOS,mobile,0,0.0,False,Low,<NA>,1
21750,3477856770102973339,1489148447,1,2017-03-10 04:20:47,1,3,10,6,4,20,47,(direct),(none),Direct,1,1,Unknown,0,0,0,0,1,0,Chrome,Linux,desktop,0,0.0,False,Low,<NA>,1


::: {.callout-note title="Notes"}

1. The final sample size used to generate the cohorts is the least number of samples (first-time visitors) to be included in each of the control and test cohorts of each audience group. These sizes come from the output of the previous step (designing a media or marketing experiment - [1](https://blog.hubspot.com/blog/tabid/6307/bid/31634/a-b-testing-in-action-3-real-life-marketing-experiments.aspx), [2](https://www.datacamp.com/blog/data-demystified-what-is-a-b-testing)).
:::

The treatment (or test) and control groups should be similar to each other for the property to be tested. This is a [fundamental requirement of test and control groups](https://www.mobileapps.com/blog/test-group#Similarities_Between_Test_and_Control_Groups). In this case, the probability (score column) is the property of interest. With this in mind, we now show selected descriptive statistics, for the score column, for both the test (treatment) and control cohorts within each desired audience group (low, medium and high propensity)

In [27]:
#| echo: true
df_aud_stats = ch.get_cohort_stats(df_infer_audience_groups)
df_aud_stats

,maudience,cohort,score_count,score_min,score_mean,score_median,score_max
0,High,Control,2206,0.093956,0.281044,0.225347,0.955219
1,High,Test,2206,0.094071,0.278541,0.225066,0.920158
2,High,<NA>,2839,0.094022,0.280057,0.226255,0.939487
3,Low,Control,2103,0.0,0.001912,0.000802,0.008468
4,Low,Test,2103,0.0,0.001858,0.000675,0.00845
5,Low,<NA>,3045,0.0,0.001973,0.000881,0.008467
6,Medium,Control,1897,0.008473,0.039143,0.033061,0.093524
7,Medium,Test,1897,0.00847,0.039329,0.033066,0.093933
8,Medium,<NA>,3456,0.008468,0.03929,0.033022,0.093914


::: {.callout-tip title="Observations"}

1. It is reassuring that there is agreement in the statistics for the probabilities per Test-Control cohort within the same audience group.
:::

Finally, the audience groups in the inference data are now briefly profiled, in order to identify characteristics that might help the marketing team build an appropriate strategy to be implemented during the campaign. For each audience group, the profile consists of the following

1. proportion of visitors who displayed the following behavior
   - whose last action during thier first visit was
     - `last_action == 'Add product(s) to cart'`
     - `last_action == 'Product detail_views'`
     - `last_action == 'Click through of product lists'`
   -  who
      - used a `medium == 'referral'` to get to the store's website on their first visit
      - added one item to their shopping cart (`added_to_cart > 0`) on their first visit
   - whose
     - first visit occurred on a weekend (`day_of_week.isin(@weekend_days)`)
2. bounce rate during all first visits
3. following descriptive statistics
   - `hour`, `day_of_week`
     - statistic: `mean`
   - `source`, `medium`, `channelGrouping`, `last_action`, `browser`, `os`, `deviceCategory`
     - statistic: `mode` (most common value)
   - `promos_displayed`, `promos_clicked`, `product_views`, `product_clicks`, `pageviews`, `added_to_cart`
     - statistics: `mean`, `max`

In [28]:
#| echo: true
df_profile = pa.get_audience_profile(df_infer_audience_groups)
with pd.option_context("display.max_columns", 1000):
    display(df_profile)

Set all specified datatypes.
Set all specified datatypes.


maudience,stat,stat_type,High,Low,Medium,column
0,hour__mean,mean,13.030892290718521,12.915735760584747,13.075586206896551,hour
1,day_of_week__mean,mean,4.009515928837402,3.9891049510412357,4.013379310344828,day_of_week
2,source__mode,mode,google,google,google,source
3,medium__mode,mode,organic,organic,organic,medium
4,channelGrouping__mode,mode,Organic Search,Organic Search,Organic Search,channelGrouping
5,last_action__mode,mode,Unknown,Unknown,Unknown,last_action
6,browser__mode,mode,Chrome,Chrome,Chrome,browser
7,os__mode,mode,Macintosh,Macintosh,Macintosh,os
8,deviceCategory__mode,mode,desktop,desktop,desktop,deviceCategory
9,hits__mean,mean,6.39167011446697,6.0924010481312925,6.590068965517242,hits


::: {.callout-note title="Notes"}

1. The audience profile consists of different types of statistics about the first visit, for each audience group.
2. The `stat` column shows the attribute from the first visit in the inference data for which a statistic is calculated.
3. The `stat_type` column indicates the type of statistic.
4. The *High*, *Medium* and *Low* columns (or just the *High* column, if the single-group audience strategy is being used) show the statistic of each attribute *within* each audience group.
:::

## Export to Disk and ML Experiment Tracking

Get the best MLFlow run ID

In [29]:
#| echo: true
best_run_id = df_deployment_candidate_mlflow_models.squeeze()["run_id"]

### Audience Cohorts

::: {.content-hidden}
Show summary of `DataFrame` with audience cohorts
:::

In [30]:
#| output: false
ut.summarize_df(df_infer_audience_groups)

,column,dtype,missing
0,fullvisitorid,string[python],0
1,visitId,string[python],0
2,visitNumber,Int8,0
3,visitStartTime,datetime64[ns],0
4,quarter,Int8,0
5,month,Int8,0
6,day_of_month,Int8,0
7,day_of_week,Int8,0
8,hour,Int8,0
9,minute,Int8,0


::: {.callout-note title="Notes"}

1. The `cohort` column has missing values for visitors who were not assigned to either the test or control groups. This is expected.
:::

Export to disk and log exported file as MLFlow artifact

In [31]:
#| echo: true
#| output: false
ut.export_and_track(
    os.path.join(
        processed_data_dir,
        f"audience_cohorts__run_"
        f"{best_run_id}__"
        f"infer_month_{month_name[1:][X_infer['month'].iloc[-1] - 1]}__"
        f"{datetime.now().strftime('%Y%m%d_%H%M%S')}.parquet.gzip",
    ),
    df_infer_audience_groups,
    (
        "inference audience cohorts for "
        f"{month_name[1:][X_infer['month'].iloc[-1] - 1]}"
    ),
    best_run_id,
)

Exported inference audience cohorts for March to file audience_cohorts__run_742cf152d5094be0b686edf910c7d398__infer_month_March__20230605_232410.parquet.gzip
Logged inference audience cohorts for March as artifact in file audience_cohorts__run_742cf152d5094be0b686edf910c7d398__infer_month_March__20230605_232410.parquet.gzip


### Audience Profiles

::: {.content-hidden}
Show summary `DataFrame` with audience profiles
:::

In [32]:
#| output: false
ut.summarize_df(df_profile)

,maudience,dtype,missing
0,stat,string[python],0
1,stat_type,string[python],0
2,High,string[python],0
3,Low,string[python],0
4,Medium,string[python],0
5,column,string[python],7


::: {.callout-note title="Notes"}

1. The behavioral attributes in the profile are not specific to an individual column. So, the `column` in the profile `DataFrame` has missing values for these attributes.
:::

Export to disk and log exported file as MLFlow artifact

In [33]:
#| echo: true
#| output: false
ut.export_and_track(
    os.path.join(
        processed_data_dir,
        f"audience_profiles__run_"
        f"{best_run_id}__"
        f"infer_month_{month_name[1:][X_infer['month'].iloc[-1] - 1]}__"
        f"{datetime.now().strftime('%Y%m%d_%H%M%S')}.parquet.gzip",
    ),
    df_profile,
    (
        "inference audience profiles for "
        f"{month_name[1:][X_infer['month'].iloc[-1] - 1]}"
    ),
    best_run_id,
)

Exported inference audience profiles for March to file audience_profiles__run_742cf152d5094be0b686edf910c7d398__infer_month_March__20230605_232413.parquet.gzip
Logged inference audience profiles for March as artifact in file audience_profiles__run_742cf152d5094be0b686edf910c7d398__infer_month_March__20230605_232413.parquet.gzip


## Next Step

The next step will add to the audience profiles, by exploring the best ML model's predictions of propensity for first-time visitors to the store during the unseen (inference) data period.